In [1]:
import os
from pathlib import Path
import sys

# !IMPORTANT!, else Hydra isnt working !
current_path = os.getcwd()
sys.path.insert(0, Path(current_path).absolute().parents[0].as_posix())

import cv2
import tensorflow_datasets as tfds
import tensorflow as tf
import tqdm
import mediapy
import numpy as np
import torch
from uha import make_pytorch_oxe_iterable_dataset, get_octo_dataset_tensorflow, get_single_dataset_tensorflow
from torchvision.transforms.functional import convert_image_dtype
import omegaconf
from omegaconf import OmegaConf

/home/marcelr/miniforge3/envs/uha_test_policy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-11 08:50:07.357970: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 08:50:07.358020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 08:50:07.359959: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 08:50:07.368753: I tensorflow/core/platform/cpu_feature_guard.cc:182] This

## Load Data
Next, we will load a trajectory from the Bridge dataset. We will use the one defined in uha/data/oxe/oxe_dataset_configs.py.

In [2]:
cfg = omegaconf.OmegaConf.load("../uha/data/conf/uha_default_load_config.yaml")
del cfg.interleaved_dataset_cfg.frame_transform_kwargs.image_augment_kwargs # comment out to get augmented data
cfg_transforms = omegaconf.OmegaConf.load("../uha/data/conf/transforms/oxe_no_remapping.yaml")

In [3]:
dataset = get_octo_dataset_tensorflow(cfg, train=True)
# dataset = get_single_dataset_tensorflow(cfg, train=True).repeat().unbatch()
is_single_dataset = False
batch_size = 128
# create Pytorch Train Dataset
dataloader = make_pytorch_oxe_iterable_dataset(dataset, train=True, batch_size=batch_size, transform_dict=cfg_transforms, num_workers=0, pin_memory=True, is_single_dataset=is_single_dataset, main_process=False)
print("len in colab", len(dataloader.dataset))
it = iter(dataloader)
batch = next(it)
images, image_wrist = [], []
for i in range(batch_size):
  images.append(batch["observation"]["image_primary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  image_wrist.append(batch["observation"]["image_secondary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]

mediapy.show_video(images, fps=10)
mediapy.show_video(image_wrist, fps=10)

2024-07-11 08:50:12.288381: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".
2024-07-11 08:50:15.046679: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Cause: Unable to locate the source code of <function _gcd_import at 0x7f2f21993400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7f2f21993400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7f2f21993400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-07-11 08:50:16.995227: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:20.446375: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:21.178881: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:23.938253: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:24.556489: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:27.628577: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:28.322953: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:31.562207: I tensorflow/


######################################################################################
# Loading the following 6 datasets (incl. sampling weight):                         #
# droid: ===================================================================0.878941 #
# robo_set: ================================================================0.085325 #
# aloha_mobile: ============================================================0.005617 #
# berkeley_mvp_converted_externally_to_rlds: ===============================0.002722 #
# berkeley_rpt_converted_externally_to_rlds: ===============================0.007823 #
# toto: ====================================================================0.019571 #
######################################################################################



2024-07-11 08:50:38.337787: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:42.557999: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:45.690783: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:48.868011: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:52.447806: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-11 08:50:55.535307: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


dataset_len called 16642183
len in colab 16642183


In [ ]:
ds = tfds.load("robo_set", 
    data_dir="gs://gresearch/robotics", split="train")

img, img_sec, img_wrist = [], [], []
for episode in ds.take(1):
    for step in episode["steps"]:
        image_ = step["observation"]["exterior_image_1_left"]
        image_secondary = step["observation"]["exterior_image_2_left"]
        wrist_image = step["observation"]["wrist_image_left"]
        action = step["action_dict"]["joint_position"]
        obs = step["observation"]["joint_position"]
        gripper = step["action_dict"]["gripper_position"]
        instruction = step["language_instruction"]
        print("act", action.numpy())
        print("obs", obs.numpy())
        # print("gripper", gripper.numpy())
        # print(instruction.numpy())
        img.append(image_.numpy())
        img_sec.append(image_secondary.numpy())
        img_wrist.append(wrist_image.numpy())

mediapy.show_video(img, fps=10)
mediapy.show_video(img_sec, fps=10)
mediapy.show_video(img_wrist, fps=10)

2024-07-11 08:14:15.249617: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".
0.0.1
Using gs://gresearch/robotics/robo_set/0.1.0 instead.


PermissionDeniedError: Error executing an HTTP request: HTTP response code 401 with body '{
  "error": {
    "code": 401,
    "message": "Anonymous caller does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).",
    "errors": [
      {
        "message": "Anonymous caller does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).",
        "domain": "global",
        "reason": "require'
	 when uploading gs://gresearch/robotics/downloads/

: 